In [1]:
import pandas as pd

In [2]:
# Gold
df = pd.read_csv("Gold.csv",sep = ",")
df['Date Local'] = pd.to_datetime(df['Date Local'], errors='coerce')
df_jan = df[(df['Date Local'] >= '2019-01-01') & (df['Date Local'] <= '2019-01-31')]
df_jan = df_jan[['Latitude', 'Longitude', 'AQI']]
df_avg = df_jan.groupby(['Latitude', 'Longitude'], as_index=False).mean()
df_avg.to_csv("Gold_clean.csv", index=False)

/var/folders/9n/7bxqt9q92tl6lchpfwnymvjh0000gn/T/ipykernel_5704/256820174.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Gold.csv",sep = ",")


In [3]:
#CMAQ
df3 = pd.read_csv("CMAQ.csv", skiprows=2)
df3 = df3.iloc[1:]
df4 = df3[['latitude', 'longitude', 'PM25_AVG']].copy()
df4['latitude'] = pd.to_numeric(df4['latitude'], errors='coerce').round(6)
df4['longitude'] = pd.to_numeric(df4['longitude'], errors='coerce').round(6)
df4['PM25_AVG'] = pd.to_numeric(df4['PM25_AVG'], errors='coerce')
df_avg2 = df4.groupby(['latitude', 'longitude'], as_index=False).mean()
df_avg2.to_csv("CMAQ_clean.csv", index=False)

/var/folders/9n/7bxqt9q92tl6lchpfwnymvjh0000gn/T/ipykernel_5704/3912202928.py:2: DtypeWarning: Columns (2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv("CMAQ.csv", skiprows=2)


In [3]:
# Get CVS1: 
# This is U2 Part (about 1000 data) si, Z1 and closest uj and Z2
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Load and rename
df_gold = pd.read_csv("Gold_clean.csv")
df_cmaq = pd.read_csv("CMAQ_clean.csv")

df_gold = df_gold.rename(columns={'Latitude': 'si_lat', 'Longitude': 'si_lon', 'AQI': 'Z1'})
df_cmaq = df_cmaq.rename(columns={'latitude': 'uj_lat', 'longitude': 'uj_lon', 'PM25_AVG': 'Z2'})

# Round coordinates to avoid float precision issues
df_gold['si_lat'] = df_gold['si_lat'].round(5)
df_gold['si_lon'] = df_gold['si_lon'].round(5)
df_cmaq['uj_lat'] = df_cmaq['uj_lat'].round(5)
df_cmaq['uj_lon'] = df_cmaq['uj_lon'].round(5)

# Prepare for NearestNeighbor
X_gold = df_gold[['si_lat', 'si_lon']].to_numpy()
X_cmaq = df_cmaq[['uj_lat', 'uj_lon']].to_numpy()

# Find nearest uj for each si
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
nbrs.fit(X_cmaq)
distances, indices = nbrs.kneighbors(X_gold)

# Get nearest Z2
nearest_cmaq = df_cmaq.iloc[indices.flatten()].reset_index(drop=True)
df_gold_reset = df_gold.reset_index(drop=True)

# Merge and round again just in case
df_matched = pd.concat([df_gold_reset, nearest_cmaq], axis=1)
df_matched[['uj_lat', 'uj_lon']] = df_matched[['uj_lat', 'uj_lon']].round(5)

# Final output
df_csv1 = df_matched[['si_lat', 'si_lon', 'uj_lat', 'uj_lon', 'Z1', 'Z2']]


In [4]:
df_csv1.to_csv("CSV1.csv", index=False)

In [5]:
# U1/U2: CSV2
df_cmaq = pd.read_csv("CMAQ_clean.csv")
df_cmaq = df_cmaq.rename(columns={'latitude': 'uj_lat', 'longitude': 'uj_lon', 'PM25_AVG': 'Z2'})
df_csv1 = pd.read_csv("CSV1.csv")

df_cmaq['uj_lat'] = df_cmaq['uj_lat'].round(5)
df_cmaq['uj_lon'] = df_cmaq['uj_lon'].round(5)
df_csv1['uj_lat'] = df_csv1['uj_lat'].round(5)
df_csv1['uj_lon'] = df_csv1['uj_lon'].round(5)

df_cmaq_unique = df_cmaq.drop_duplicates(subset=['uj_lat', 'uj_lon'])
df_csv1_unique = df_csv1[['uj_lat', 'uj_lon']].drop_duplicates()

df_unmatched = df_cmaq_unique.merge(
    df_csv1_unique,
    on=['uj_lat', 'uj_lon'],
    how='left',
    indicator=True
)

df_unmatched = df_unmatched[df_unmatched['_merge'] == 'left_only'].drop(columns=['_merge'])

df_unmatched.to_csv("CSV2.csv", index=False)



In [12]:
len(df_csv1) 

970

In [13]:
#Sanity check - make sure CSV2 doesn't contain any uj points from CSV1 (it shouldn't)
df_csv2 = df_unmatched.copy()
df_csv2[['uj_lat', 'uj_lon']] = df_csv2[['uj_lat', 'uj_lon']].round(5)  # Extra safeguard

# Perform an inner merge to check if there is any overlap
df_overlap = df_csv2.merge(df_csv1_unique, on=['uj_lat', 'uj_lon'], how='inner')

if not df_overlap.empty:
    print(f"❌ Error: CSV2 contains {len(df_overlap)} uj points that are also in CSV1. These should have been excluded.")
    print("Duplicated coordinates:")
    print(df_overlap[['uj_lat', 'uj_lon']].drop_duplicates())
else:
    print("✅ Passed: CSV2 does not contain any uj points from CSV1. The split is correct.")

✅ Passed: CSV2 does not contain any uj points from CSV1. The split is correct.


In [15]:
# Train/test split

from sklearn.model_selection import train_test_split
df_csv1_train, df_csv1_test = train_test_split(df_csv1, test_size=0.1, random_state=42)
df_csv1_train.to_csv("CSV1_train.csv", index=False)
df_csv1_test.to_csv("CSV1_test.csv", index=False)

In [3]:
df_csv1_test = pd.read_csv("CSV1_test.csv")

In [5]:
len(df_csv1_test)

97

In [7]:
import pandas as pd
import numpy as np

df_proj = pd.read_csv("projection_matrix_total.csv")
df_csv1_test = pd.read_csv("CSV1_test.csv")


df_proj_csv1_like = df_proj[(df_proj["Z1"] - df_proj["proj_Z1"]).abs() > 1e-8]


keys = ['uj_lat','uj_lon','Z1','Z2']
def rounded_view(df, cols=keys, nd=6):
    v = df[cols].astype(float).copy()
    for c in cols:
        v[c] = v[c].round(nd)
    return v

rv_proj_csv1_like = rounded_view(df_proj_csv1_like)         
rv_csv1_test      = rounded_view(df_csv1_test)               

# test = CSV1-like ∩ CSV1_test
df_proj_test_idx = rv_proj_csv1_like.merge(
    rv_csv1_test.drop_duplicates(), on=keys, how='inner'
).index


df_proj_test = df_proj_csv1_like.iloc[df_proj_test_idx].copy()


rv_proj_all = rounded_view(df_proj)                          
rv_test_keys = rounded_view(df_proj_test).drop_duplicates()  

anti = rv_proj_all.merge(rv_test_keys, on=keys, how='left', indicator=True)
keep_mask = anti['_merge'].eq('left_only').to_numpy()


df_proj_train = df_proj.loc[keep_mask].copy()


assert len(df_proj) == len(df_proj_train) + len(df_proj_test), \
    f"mismatch: total={len(df_proj)}, train={len(df_proj_train)}, test={len(df_proj_test)}"


df_proj_train.to_csv("projection_matrix_train.csv", index=False)
df_proj_test.to_csv("projection_matrix_test.csv", index=False)


In [8]:
len(df_proj_train)

50873

In [9]:
len(df_proj_test)

97